In [ ]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_predict, GridSearchCV, cross_val_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics

In [ ]:
def setpvalue(series):
    return (series<0.01).replace({True:'*',False:''}) + (series<0.05).replace({True:'*',False:''}) + (series<0.1).replace({True:'*',False:''})

# Linear Regression

In [ ]:
import scipy.stats as sp # for spearman correlation
import statsmodels.api as sm
from scipy import stats # for logistic regression
from statsmodels.formula.api import ols

In [ ]:
def analysis(year,spearmanflag):   
   
    # Read raw file
    raw = pd.read_csv('dataset'+str(year)+'.csv') 
    
    # Fill missing value: years, ages with mean
    for columnname in raw.iloc[:,7:15].columns:
        meanvalue = raw[raw[columnname] !=0][columnname].mean()
        raw[columnname] = raw[columnname].replace({0:meanvalue})
    
    
    dataset_X = pd.concat([raw.iloc[:,7],raw.iloc[:,9:11], raw.iloc[:,12:-4],raw.iloc[:,-2:]], axis = 1)
    dataset_y = raw.iloc[:,2]
    
    glm = sm.OLS(dataset_y, dataset_X)
    resultmodel = glm.fit()
    
    coeffandp = pd.concat([pd.DataFrame(resultmodel.params, columns = [year]),\
                           pd.DataFrame(resultmodel.pvalues, columns = ['pvalue'])] , axis  = 1)
    
    
    
    coeffandp['sx'] = np.where(coeffandp.pvalue < 0.10, '*', '')
    coeffandp['s5'] = np.where(coeffandp.pvalue < 0.05, '*', '')
    coeffandp['s1'] = np.where(coeffandp.pvalue < 0.01, '*', '')
    coeffandp[str(year)+'mk'] = coeffandp['sx'] + coeffandp['s5'] + coeffandp['s1']
    
    coeffandp.drop(['sx','s5','s1','pvalue'], inplace = True, axis =1)
    
    if (spearmanflag == 1):
        
        fulldata = pd.concat([dataset_y,dataset_X], axis = 1)

        spearmanresult = sp.spearmanr(fulldata)

        spearmandf = pd.DataFrame(spearmanresult.correlation, index = fulldata.columns)
        spearmandf.columns = fulldata.columns
        spearmandfp = pd.DataFrame(spearmanresult.pvalue, index = fulldata.columns)
        spearmandfp.columns = fulldata.columns
                                      
        return coeffandp, resultmodel, spearmandf, spearmandfp
    else:
        return coeffandp, resultmodel
                                      
                                

        
    

In [ ]:
coefdict = {}
modeldict = {}
spearmandict = {}

for startingyear in range(960,1280,10):

    coeffdf, model, spearmandf, spearmandfp = analysis(startingyear,1)
    
    coefdict[startingyear] = coeffdf
    modeldict[startingyear] = model
    spearmandict[startingyear] = (spearmandf,spearmandfp)

### R-square

In [ ]:
dflist = []

for year in range(960,1280,10):
    stats ={}
    stats['R-squared'] = modeldict[year].rsquared
    stats['Adjusted R-squared'] = modeldict[year].rsquared_adj
    stats['F-statistic'] = modeldict[year].fvalue
    stats['F-stat p-value'] = modeldict[year].f_pvalue
    
    dflist.append(pd.DataFrame(stats,index =[year]))

In [ ]:
# R-squared table

significantvariablenumber = {}
for startingyear in range(960,1280,10):
    significantvariablenumber[startingyear] = len(coefdict[startingyear][coefdict[startingyear].iloc[:,-1] != ''])
    
    
rsquare = pd.concat(dflist)
rsquare.iloc[:,-1] = setpvalue(rsquare.iloc[:,-1])
pd.concat([rsquare,pd.DataFrame(significantvariablenumber, index = ['Number of Variable found significant']).T],axis =1).to_csv('newrsquare.csv')

###  Geographic stats

In [ ]:
significantlu  ={}
for startingyear in range(960,1280,10):
    tempdf = coefdict[startingyear][6:35]
    significantlu[startingyear] = len(tempdf[tempdf.iloc[:,-1] != ''])

In [ ]:
dflist = []
for startingyear in range(960,1280,10):
    
    convertingdict = {}

    convertingdict['Longitude'] = coefdict[startingyear].iloc[4:6].iloc[0,0]
    convertingdict['Lonp'] = coefdict[startingyear].iloc[4:6].iloc[0,1]
    convertingdict['Latitude'] = coefdict[startingyear].iloc[4:6].iloc[1,0]
    convertingdict['Ltp'] = coefdict[startingyear].iloc[4:6].iloc[1,1]

    dflist.append(pd.DataFrame(convertingdict,index = [startingyear]))
lonlatdf = pd.concat(dflist)

In [ ]:
geo = pd.concat([pd.DataFrame(significantlu, index = ['Number of Significant Lu']).T,lonlatdf],axis = 1)

In [ ]:
geo.to_csv('georesult.csv')

### Bloodline

In [ ]:
bloodroutes = ['entry_8','entry_59','entry_60','entry_62','entry_118']

In [ ]:
for startingyear in range(960,1280,10):
    print (coefdict[startingyear].T[bloodroutes])

In [ ]:
dflist = []

for startingyear in range(960,1280,10):
    tempdf = coefdict[startingyear].T[bloodroutes].T
    convertingdict ={}
    for i in range(len(bloodroutes)):
        convertingdict[bloodroutes[i]] = tempdf.iloc[i,0]
        convertingdict[bloodroutes[i]+'p'] = tempdf.iloc[i,1]

    dflist.append( pd.DataFrame(convertingdict, index = [startingyear]))

In [ ]:
pd.concat(dflist).to_csv('bloodresult.csv')

### Jinshi

In [ ]:
eliteroutes = ['entry_36','entry_165']

In [ ]:
dflist = []

for startingyear in range(960,1280,10):
    tempdf = coefdict[startingyear].T[eliteroutes].T
    convertingdict ={}
    for i in range(len(eliteroutes)):
        convertingdict[eliteroutes[i]] = tempdf.iloc[i,0]
        convertingdict[eliteroutes[i]+'p'] = tempdf.iloc[i,1]

    dflist.append( pd.DataFrame(convertingdict, index = [startingyear]))

In [ ]:
pd.concat(dflist).to_csv('eliteresult.csv')

### Network

In [ ]:
network = ['status','nemesis']

In [ ]:
dflist = []

for startingyear in range(960,1280,10):
    tempdf = coefdict[startingyear].T[network].T
    convertingdict ={}
    for i in range(len(network)):
        convertingdict[network[i]] = tempdf.iloc[i,0]
        convertingdict[network[i]+'p'] = tempdf.iloc[i,1]

    dflist.append( pd.DataFrame(convertingdict, index = [startingyear]))

In [ ]:
pd.concat(dflist).to_csv('networkresult.csv')

# Random Forest

In [ ]:
conninput = sqlite3.connect("edgelist.db")

In [ ]:
def randomforestandreturn(year):
    
    # Read raw file
    raw = pd.read_csv('dataset'+str(year)+'.csv') 
    
    # Fill missing value: years, ages with mean
    for columnname in raw.iloc[:,7:15].columns:
        meanvalue = raw[raw[columnname] !=0][columnname].mean()
        raw[columnname] = raw[columnname].replace({0:meanvalue})
    
    
    dataset_X = pd.concat([raw.iloc[:,7],raw.iloc[:,9:11], raw.iloc[:,12:-4],raw.iloc[:,-2:]], axis = 1)
    dataset_y = raw.iloc[:,2]

    # Perform Grid-Search 
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,20),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(dataset_X, dataset_y)
    best_params = grid_result.best_params_

    sel = SelectFromModel(RandomForestRegressor(max_depth=best_params["max_depth"],
                                n_estimators=best_params["n_estimators"],
                                random_state=False, verbose=False))
    sel.fit(dataset_X, dataset_y)

    rfr = RandomForestRegressor(max_depth=best_params["max_depth"],
                                n_estimators=best_params["n_estimators"],
                                random_state=False, verbose=False)
    # Perform K-Fold CV
    scores = cross_val_score(rfr, dataset_X, dataset_y, cv=10, scoring='neg_mean_absolute_error')
    predictions = cross_val_predict(rfr, dataset_X,dataset_y, cv=10)
    MSE = metrics.mean_squared_error(dataset_y, predictions)

    #    return predictions
    
    rfr.fit(dataset_X, dataset_y)
    
    return MSE, pd.DataFrame(rfr.feature_importances_, index = dataset_X.columns, columns = [year]).T

In [ ]:
connoutput =  sqlite3.connect("gini_finalR.db")

In [ ]:
msedict = {}
ginidict = {}

In [ ]:
for startingyear in range(960,1280,10):
    
    if startingyear in msedict:
        continue
    
    print('processing '+str(startingyear))
    
    mse, gini = randomforestandreturn(startingyear)

    #gini.to_sql('gini'+str(startingyear), con=connoutput, if_exists='replace')
    
    msedict[startingyear] = mse
    ginidict[startingyear] = gini

In [ ]:
msetable = pd.DataFrame(msedict, ['mse']).T
    

In [ ]:
msetable['sd'] = np.sqrt(msetable['mse'])

In [ ]:
msetable.to_csv('newmse.csv')

In [ ]:
convertingdict ={}

for startingyear in range(960,1280,10):

    convertingdict[startingyear] = ginidict[startingyear].T.sort_values(by = startingyear, ascending = False).index[0:10]


In [ ]:
pd.DataFrame(convertingdict, index = range(1,11)).T.to_csv('giniresult.csv')

In [ ]:
pd.DataFrame(msedict, index = ['mse']).T.to_csv('mse.csv')

In [ ]:
ginis = {}
for year in range(960,1280,10):
    ginis[year]= pd.read_sql_query("SELECT * FROM gini"+str(year), 
                         sqlite3.connect('gini_finalR.db')).drop_duplicates()

In [ ]:
dflis = []
for year in range(960,1280,10):

    tempdf = ginis[year].T.sort_values(by = 0, ascending = False).iloc[1:11].reset_index()
    tempdf.columns = [str(year)+'+factor',str(year)+'+gini']
    dflis.append(tempdf)

In [ ]:
pd.concat(dflis, axis =1).to_csv('ginipresent10.csv')

### Coefficient

In [ ]:
dflist = []
for year in coefdict:
    dflist.append(coefdict[year])

In [ ]:
coeffull = pd.concat(dflist, axis = 1)

In [ ]:
coeffull.to_csv('coefficient.csv')

In [ ]:
pearsondict = {}

for year in range(960,1280,10):

    raw = pd.read_csv('dataset'+str(year)+'.csv') 
    
    pearsondict[year]=stats.pearsonr(raw.iloc[:,-2],raw.iloc[:,-1])

In [ ]:
pd.DataFrame(pearsondict, index = ['corr','p-value']).T.to_csv('nemesis-status-correlation.csv')

In [ ]:
for year in range(960,1280,10):

    raw = pd.read_csv('dataset'+str(year)+'.csv') 
    
    print(raw.iloc[:,-2])
    print (raw.iloc[:,-1])